In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

### Inflation Forecasts

In [2]:
# Get IMF Data
    # Go to: https://www.imf.org/en/Publications/WEO/frequently-asked-questions
    # Related Links -> Historical WEO Forecasts Database
    # Save in same folder with code

In [3]:
df = pd.read_excel('WEOhistorical.xlsx', sheet_name='pcpi_pch')

In [4]:
# Clean dataframe
countries = ['Euro area','Japan','United Kingdom','China','Australia','Canada','Switzerland','Sweden','Korea','Norway',
'New Zealand','India','Mexico','South Africa','Poland','Israel','Indonesia','Czech Republic','Türkiye','Hungary',
'Chile','Russia','United States','Brazil','Thailand','Philippines','Malaysia','Colombia','Romania','Peru',
'Germany','France','Italy','Spain','Denmark']
df = df[df['country'].isin(countries)]
currencies = {'Euro area':'EUR','Japan':'JPY','United Kingdom':'GBP','China':'CNY','Australia':'AUD','Canada':'CAD',
 'Switzerland':'CHF','Sweden':'SEK','Korea':'KRW','Norway':'NOK','New Zealand':'NZD','India':'INR',
 'Mexico':'MXN','South Africa':'ZAR','Poland':'PLN','Israel':'ILS','Indonesia':'IDR','Czech Republic':'CZK',
 'Türkiye':'TRY','Hungary':'HUF','Chile':'CLP','Russia':'RUB','United States':'USD','Brazil':'BRL',
 'Thailand':'THB','Philippines':'PHP','Malaysia':'MYR','Colombia':'COP','Romania':'RON','Peru':'PEN',
 'Germany':'DEM','France':'FRF','Italy':'ITL','Spain':'ESP','Denmark':'DKK'}
df['currency'] = df['country'].map(currencies)
currency_column = df.pop('currency')
df.insert(0, 'currency', currency_column)
df.drop(columns=['country','WEO_Country_Code','ISOAlpha_3Code'], inplace=True)
df = pd.melt(df, id_vars=['currency', 'year'], var_name='forecast_date', value_name='inflation_forecast')
df['forecast_date'] = df['forecast_date'].str.replace('pcpi_pch', '')

In [5]:
# Convert to dates
def convert_date(date_str):
    if date_str.startswith('S'):
        return f'06/30/{date_str[1:]}'
    elif date_str.startswith('F'):
        return f'12/31/{date_str[1:]}'
    else:
        return date_str
df['forecast_date'] = df['forecast_date'].apply(convert_date)
df['inflation_forecast'].replace('.', np.nan, inplace=True)
df = df.dropna(subset=['inflation_forecast'])
df.rename(columns={'year': 'realized_date'}, inplace=True)
df['realized_date'] = pd.to_datetime(df['realized_date'].astype(str) + '-12-31')
df['forecast_date'] = pd.to_datetime(df['forecast_date'])
# Reorder columns
new_order = ['forecast_date','realized_date', 'currency', 'inflation_forecast']
df = df[new_order]
# Keep only the forecasts for the next 3 years
df['forecast_year'] = pd.to_datetime(df['forecast_date']).dt.year
df['realized_year'] = pd.to_datetime(df['realized_date']).dt.year
df = df[(df['realized_year'] == df['forecast_year']) | (df['realized_year'] == df['forecast_year'] + 1) | 
        (df['realized_year'] == df['forecast_year'] + 2)]
df = df.drop(columns=['forecast_year'])
df = df.drop(columns=['realized_year'])

In [6]:
# Create new dataframe
date_range = pd.date_range(start='1990-06-30', end='2024-06-30', freq='Q')
currency_tickers = ['EUR','JPY','GBP','CNY','AUD','CAD','CHF','SEK','KRW','NOK','NZD','INR','MXN','ZAR',
                    'PLN','ILS','IDR','CZK','TRY','HUF','CLP','RUB','USD','BRL','THB','PHP','MYR','COP',
                    'RON','PEN','DEM','FRF','ITL','ESP','DKK']
df2 = pd.DataFrame(index=date_range, columns=currency_tickers)
df2 = df2.fillna('')

In [7]:
# Fill new dataframe with forecasts
for dt in date_range:
    for ccy in currency_tickers:
        if dt.month == 3:
            x = 0.75
            fcst_date = datetime.date(dt.year-1, 12, 31)
        elif dt.month == 6:
            x = 0.5
            fcst_date = datetime.date(dt.year, 6, 30)
        elif dt.month == 9:
            x = 0.25
            fcst_date = datetime.date(dt.year, 6, 30)
        else:
            x = 0
            fcst_date = datetime.date(dt.year, 12, 31)
            
        # Query for this_year
        this_year_df = df[(df['currency'] == ccy) & 
                          (df['forecast_date'] == fcst_date.strftime('%Y-%m-%d')) & 
                          (df['realized_date'] == datetime.date(dt.year, 12, 31).strftime('%Y-%m-%d'))] 
        if not this_year_df.empty:
            this_year = this_year_df['inflation_forecast'].iloc[0]
        else:
            this_year = np.nan  # Set a default value when the DataFrame is empty
        
        # Query for next_year
        next_year_df = df[(df['currency'] == ccy) & 
                          (df['forecast_date'] == fcst_date.strftime('%Y-%m-%d')) & 
                          (df['realized_date'] == datetime.date(dt.year+1, 12, 31).strftime('%Y-%m-%d'))]
        if not next_year_df.empty:
            next_year = next_year_df['inflation_forecast'].iloc[0]
        else:
            next_year = np.nan  # Set a default value when the DataFrame is empty
        
        df2.loc[dt, ccy] = x * this_year + (1-x) * next_year
df2

,EUR,JPY,GBP,CNY,AUD,CAD,CHF,SEK,KRW,NOK,...,PHP,MYR,COP,RON,PEN,DEM,FRF,ITL,ESP,DKK
1990-06-30,NaN,1.798074,7.566062,5.999994,5.888113,5.070405,3.599999,7.113323,4.850003,4.05,...,8.243792,3.499694,20.999997,0.784324,1953.699528,2.948221,2.950001,5.300001,5.190627,3.399997
1990-09-30,NaN,1.533158,7.258211,5.749994,5.432163,5.188389,3.400003,7.669982,4.425004,3.975002,...,7.115959,3.487432,20.499992,0.782784,1092.949793,2.977582,2.875002,4.999998,4.843957,3.349997
1990-12-31,NaN,2.318297,6.648391,6.000006,6.138362,6.134514,4.20718,8.315347,5.999994,4.207193,...,7.490216,3.475169,19.999987,0.781245,232.200057,3.695593,3.0,5.600006,6.712165,2.954689
1991-03-31,NaN,2.301222,5.983761,5.750003,5.885897,5.600885,3.879787,7.18569,5.499997,4.129545,...,7.018146,3.497401,19.49999,0.779732,189.750045,3.62965,2.925001,5.425002,6.382904,2.877979
1991-06-30,NaN,3.580518,4.949996,11.55,5.580229,4.429107,3.85,7.528264,8.250001,4.051898,...,11.148662,3.519633,18.999992,70.685644,101.85,3.318882,2.999999,5.881185,5.75,2.801269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,4.125493,2.461387,4.910069,2.087822,4.273235,3.142085,2.02009,4.580555,2.920183,3.85,...,4.788009,2.972822,8.144615,8.137527,4.059381,4.634911,3.754136,3.511385,3.728216,3.8
2023-09-30,3.527145,2.32551,3.944501,2.139279,3.735659,2.763616,1.807744,3.462378,2.609346,3.325,...,4.013526,3.014745,6.790115,6.967965,3.242372,3.856002,3.108754,3.041239,3.440251,3.3
2023-12-31,3.251252,2.884505,3.650762,1.691156,3.980015,2.432314,1.983835,3.614193,2.298246,3.669267,...,3.22192,2.673335,5.24299,5.784575,2.912004,3.532008,2.46,2.608474,3.928783,2.8
2024-03-31,2.977311,2.648004,3.253595,1.809483,3.826812,2.305707,1.905338,3.385645,2.223684,3.40195,...,3.155023,2.581197,4.82438,5.237242,2.69925,3.198547,2.333921,2.496821,3.461866,2.625


### Growth Forecasts

In [8]:
df = pd.read_excel('WEOhistorical.xlsx', sheet_name='ngdp_rpch')

# Clean dataframe
countries = ['Euro area','Japan','United Kingdom','China','Australia','Canada','Switzerland','Sweden','Korea','Norway',
'New Zealand','India','Mexico','South Africa','Poland','Israel','Indonesia','Czech Republic','Türkiye','Hungary',
'Chile','Russia','United States','Brazil','Thailand','Philippines','Malaysia','Colombia','Romania','Peru',
'Germany','France','Italy','Spain','Denmark']
df = df[df['country'].isin(countries)]
currencies = {'Euro area':'EUR','Japan':'JPY','United Kingdom':'GBP','China':'CNY','Australia':'AUD','Canada':'CAD',
 'Switzerland':'CHF','Sweden':'SEK','Korea':'KRW','Norway':'NOK','New Zealand':'NZD','India':'INR',
 'Mexico':'MXN','South Africa':'ZAR','Poland':'PLN','Israel':'ILS','Indonesia':'IDR','Czech Republic':'CZK',
 'Türkiye':'TRY','Hungary':'HUF','Chile':'CLP','Russia':'RUB','United States':'USD','Brazil':'BRL',
 'Thailand':'THB','Philippines':'PHP','Malaysia':'MYR','Colombia':'COP','Romania':'RON','Peru':'PEN',
 'Germany':'DEM','France':'FRF','Italy':'ITL','Spain':'ESP','Denmark':'DKK'}
df['currency'] = df['country'].map(currencies)
currency_column = df.pop('currency')
df.insert(0, 'currency', currency_column)
df.drop(columns=['country','WEO_Country_Code','ISOAlpha_3Code'], inplace=True)
df = pd.melt(df, id_vars=['currency', 'year'], var_name='forecast_date', value_name='growth_forecast')
df['forecast_date'] = df['forecast_date'].str.replace('ngdp_rpch', '')

# Convert to dates
def convert_date(date_str):
    if date_str.startswith('S'):
        return f'06/30/{date_str[1:]}'
    elif date_str.startswith('F'):
        return f'12/31/{date_str[1:]}'
    else:
        return date_str
df['forecast_date'] = df['forecast_date'].apply(convert_date)
df['growth_forecast'].replace('.', np.nan, inplace=True)
df = df.dropna(subset=['growth_forecast'])
df.rename(columns={'year': 'realized_date'}, inplace=True)
df['realized_date'] = pd.to_datetime(df['realized_date'].astype(str) + '-12-31')
df['forecast_date'] = pd.to_datetime(df['forecast_date'])
# Reorder columns
new_order = ['forecast_date','realized_date', 'currency', 'growth_forecast']
df = df[new_order]
# Keep only the forecasts for the next 3 years
df['forecast_year'] = pd.to_datetime(df['forecast_date']).dt.year
df['realized_year'] = pd.to_datetime(df['realized_date']).dt.year
df = df[(df['realized_year'] == df['forecast_year']) | (df['realized_year'] == df['forecast_year'] + 1) | 
        (df['realized_year'] == df['forecast_year'] + 2)]
df = df.drop(columns=['forecast_year'])
df = df.drop(columns=['realized_year'])

# Create new dataframe
date_range = pd.date_range(start='1990-06-30', end='2024-06-30', freq='Q')
currency_tickers = ['EUR','JPY','GBP','CNY','AUD','CAD','CHF','SEK','KRW','NOK','NZD','INR','MXN','ZAR',
                    'PLN','ILS','IDR','CZK','TRY','HUF','CLP','RUB','USD','BRL','THB','PHP','MYR','COP',
                    'RON','PEN','DEM','FRF','ITL','ESP','DKK']
df3 = pd.DataFrame(index=date_range, columns=currency_tickers)
df3 = df3.fillna('')

# Fill new dataframe with forecasts
for dt in date_range:
    for ccy in currency_tickers:
        if dt.month == 3:
            x = 0.75
            fcst_date = datetime.date(dt.year-1, 12, 31)
        elif dt.month == 6:
            x = 0.5
            fcst_date = datetime.date(dt.year, 6, 30)
        elif dt.month == 9:
            x = 0.25
            fcst_date = datetime.date(dt.year, 6, 30)
        else:
            x = 0
            fcst_date = datetime.date(dt.year, 12, 31)
            
        # Query for this_year
        this_year_df = df[(df['currency'] == ccy) & 
                          (df['forecast_date'] == fcst_date.strftime('%Y-%m-%d')) & 
                          (df['realized_date'] == datetime.date(dt.year, 12, 31).strftime('%Y-%m-%d'))] 
        if not this_year_df.empty:
            this_year = this_year_df['growth_forecast'].iloc[0]
        else:
            this_year = np.nan  # Set a default value when the DataFrame is empty
        
        # Query for next_year
        next_year_df = df[(df['currency'] == ccy) & 
                          (df['forecast_date'] == fcst_date.strftime('%Y-%m-%d')) & 
                          (df['realized_date'] == datetime.date(dt.year+1, 12, 31).strftime('%Y-%m-%d'))]
        if not next_year_df.empty:
            next_year = next_year_df['growth_forecast'].iloc[0]
        else:
            next_year = np.nan  # Set a default value when the DataFrame is empty
        
        df3.loc[dt, ccy] = x * this_year + (1-x) * next_year
df3

,EUR,JPY,GBP,CNY,AUD,CAD,CHF,SEK,KRW,NOK,...,PHP,MYR,COP,RON,PEN,DEM,FRF,ITL,ESP,DKK
1990-06-30,NaN,4.321616,1.656155,5.249997,2.296177,2.331636,2.165679,1.026519,6.560595,2.096906,...,5.990725,6.608982,3.75412,1.475676,2.249897,3.1066,3.061984,2.945202,3.872594,1.650001
1990-09-30,NaN,4.279198,1.911415,5.124994,2.531566,2.698509,2.222589,0.988178,6.980481,2.233838,...,6.249541,6.554473,3.878099,1.537838,3.875618,2.89639,3.057621,2.918402,4.027288,1.825002
1990-12-31,NaN,3.693228,1.301691,4.999997,1.640835,1.076808,2.039074,0.663182,7.600124,3.696829,...,5.366726,6.503103,4.120393,1.943767,5.501339,3.297073,3.027998,2.679869,3.298409,2.166016
1991-03-31,NaN,3.819919,1.588113,4.874997,2.171664,1.582606,2.115901,0.872387,7.625077,3.605976,...,5.468168,6.338637,4.119528,1.863007,5.250893,3.275638,3.053352,2.729546,3.191981,2.113985
1991-06-30,NaN,3.725562,-0.136679,5.250001,2.050004,1.224165,1.539779,-0.050182,7.075752,3.515123,...,2.566498,6.174171,4.118664,2.499319,4.506932,2.353012,2.390141,2.125251,2.626914,2.061954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,1.077682,1.160554,0.35098,4.859455,1.646727,1.475646,1.312795,0.265838,1.971933,2.34234,...,5.920349,4.504999,1.442355,3.025004,2.698687,0.494089,0.977324,0.715878,1.739801,0.5
2023-09-30,1.222162,1.089689,0.651918,4.670343,1.672232,1.486928,1.55615,0.639052,2.205466,2.441246,...,5.863904,4.517057,1.683224,3.337502,2.848315,0.795848,1.115581,0.747717,1.854558,0.75
2023-12-31,1.225959,1.033159,0.636035,4.159791,1.237469,1.607917,1.799219,0.618702,2.217221,1.499105,...,5.879851,4.272987,1.977707,3.78972,2.743546,0.914864,1.345776,0.655126,1.660131,1.4
2024-03-31,1.370706,0.937255,0.972733,4.149056,1.422492,1.800328,1.649414,1.053565,2.228527,1.432186,...,5.934116,4.307273,2.21578,3.802551,2.832659,1.184486,1.462093,0.752366,1.765061,1.35


### To Excel

In [9]:
df2.index.name = 'date'
df3.index.name = 'date'
with pd.ExcelWriter('WEO_forecasts.xlsx') as writer:
    df2.to_excel(writer, sheet_name='cpi')
    df3.to_excel(writer, sheet_name='gdp')